## 1. Imports and Setup

In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append("../src")

from strategy import Strategy
from backtest import run_backtest

# 2. Load and clean data

In [2]:
# Load your BTC/USD hourly data
data_path = '../data/btc_hour.csv'
df = pd.read_csv(data_path)
df.head()

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,2011-11-28 00:00:00+00:00,2.499,2.461,2.480,3457.45,8571.74,2.497,direct,NaN
1,2011-11-28 01:00:00+00:00,2.498,2.480,2.497,681.46,1698.15,2.498,direct,NaN
2,2011-11-28 02:00:00+00:00,2.499,2.485,2.498,624.29,1559.13,2.487,direct,NaN
3,2011-11-28 03:00:00+00:00,2.498,2.442,2.487,2648.16,6519.05,2.442,direct,NaN
4,2011-11-28 04:00:00+00:00,2.500,2.440,2.442,4299.98,10667.19,2.462,direct,NaN


In [3]:
df

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,2011-11-28 00:00:00+00:00,2.499,2.461,2.480,3457.45,8.571740e+03,2.497,direct,NaN
1,2011-11-28 01:00:00+00:00,2.498,2.480,2.497,681.46,1.698150e+03,2.498,direct,NaN
2,2011-11-28 02:00:00+00:00,2.499,2.485,2.498,624.29,1.559130e+03,2.487,direct,NaN
3,2011-11-28 03:00:00+00:00,2.498,2.442,2.487,2648.16,6.519050e+03,2.442,direct,NaN
4,2011-11-28 04:00:00+00:00,2.500,2.440,2.442,4299.98,1.066719e+04,2.462,direct,NaN
...,...,...,...,...,...,...,...,...,...
118054,2025-05-16 22:00:00+00:00,103649.830,103450.360,103643.590,978.41,1.012609e+08,103551.320,direct,NaN
118055,2025-05-16 23:00:00+00:00,103718.440,103465.470,103551.320,312.40,3.235507e+07,103499.600,direct,NaN
118056,2025-05-17 00:00:00+00:00,103569.600,103141.310,103499.600,488.11,5.044067e+07,103388.020,direct,NaN
118057,2025-05-17 01:00:00+00:00,103388.090,102642.390,103388.020,669.72,6.891526e+07,102859.210,direct,NaN


In [4]:
# Data cleaning: keep only time, close, volumeto; cast time to datetime; rename volumeto to volume
df = df[['time', 'close', 'volumeto']].copy()
df['time'] = pd.to_datetime(df['time'])
df = df.rename(columns={'volumeto': 'volume'})
df.head()

,time,close,volume
0,2011-11-28 00:00:00+00:00,2.497,8571.74
1,2011-11-28 01:00:00+00:00,2.498,1698.15
2,2011-11-28 02:00:00+00:00,2.487,1559.13
3,2011-11-28 03:00:00+00:00,2.442,6519.05
4,2011-11-28 04:00:00+00:00,2.462,10667.19


In [5]:
df.tail()

,time,close,volume
118054,2025-05-16 22:00:00+00:00,103551.32,1.012609e+08
118055,2025-05-16 23:00:00+00:00,103499.60,3.235507e+07
118056,2025-05-17 00:00:00+00:00,103388.02,5.044067e+07
118057,2025-05-17 01:00:00+00:00,102859.21,6.891526e+07
118058,2025-05-17 02:00:00+00:00,103307.02,2.715254e+07


# 3. Define strategy

### Buy and Hold strategy

In [6]:
class BuyAndHoldStrategy(Strategy):
    def __init__(self, initial_capital=10000):
        super().__init__(initial_capital)
        self.has_bought = False

    def process_bar(self, bar):
        self.current_bar = bar

    def get_signal(self):
        if not self.has_bought:
            self.has_bought = True
            return 'buy'
        return 'hold'

### SMA Crossover Strategy

In [7]:
class SMACrossoverStrategy(Strategy):
    def __init__(self, initial_capital=10000, fast=20, slow=100):
        super().__init__(initial_capital)
        self.prices = []
        self.fast = fast
        self.slow = slow
        self.last_signal = 'hold'

    def process_bar(self, bar):
        self.current_bar = bar
        self.prices.append(bar['close'])
        if len(self.prices) < self.slow:
            self.last_signal = 'hold'
            return

        fast_ma = pd.Series(self.prices).rolling(self.fast).mean().iloc[-1]
        slow_ma = pd.Series(self.prices).rolling(self.slow).mean().iloc[-1]

        if fast_ma > slow_ma and self.position == 0:
            self.last_signal = 'buy'
        elif fast_ma < slow_ma and self.position == 1:
            self.last_signal = 'sell'
        else:
            self.last_signal = 'hold'

    def get_signal(self):
        return self.last_signal

# 4. Run Backtests

In [8]:
# Buy and Hold
bh_results = run_backtest(BuyAndHoldStrategy, data_path, initial_capital=10000)

# SMA Crossover
sma_results = run_backtest(
    lambda initial_capital: SMACrossoverStrategy(initial_capital, fast=20, slow=100),
    data_path,
    initial_capital=10000
)

/Users/a/Desktop/repos/ns/fork/nstrade/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:191: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


KeyError: -1